In [2]:
import os
import sys
import random
import threading
import numpy as np
import pythreejs as three
import ipywidgets as widgets
from IPython.display import display
sys.path.insert(0, os.path.abspath(".."))
from cube_simulator_full import CubeTracker, CubeColorizer

class CubeVisualizer2D_pythreejs:
    def __init__(self, figsize=(500, 400), facelet_size=30.0, gap=2.0):
        if CubeTracker is None or CubeColorizer is None:
             raise RuntimeError("Simulator classes not loaded.")

        self.colorizer = CubeColorizer()
        self.tracker = self.colorizer.cube_tracker
        self.fig_width, self.fig_height = figsize
        self.facelet_size = facelet_size
        self.gap = gap
        self.cell_size = facelet_size + gap
        # Updated scramble map to use standard HTM notation
        self.scramble_move_map = {
            0: ['F', 'F2', 'F\'', 'B', 'B2', 'B\''], # FB Axis
            1: ['U', 'U2', 'U\'', 'D', 'D2', 'D\''], # UD Axis
            2: ['R', 'R2', 'R\'', 'L', 'L2', 'L\''], # RL Axis
        }

        # --- Layout Definition ---
        self.grid_positions = {
            'Z': (3, 6),
            'x': (0, 3),
            'y': (3, 3),
            'X': (6, 3),
            'Y': (9, 3),
            'z': (3, 0),
        }
        self.min_grid_x = min(pos[0] for pos in self.grid_positions.values())
        self.max_grid_x = max(pos[0] + 3 for pos in self.grid_positions.values())
        self.min_grid_y = min(pos[1] for pos in self.grid_positions.values())
        self.max_grid_y = max(pos[1] + 3 for pos in self.grid_positions.values())
        self.layout_width_grid = self.max_grid_x - self.min_grid_x
        self.layout_height_grid = self.max_grid_y - self.min_grid_y
        self.layout_width_px = self.layout_width_grid * self.cell_size
        self.layout_height_px = self.layout_height_grid * self.cell_size
        self.center_x_px = (self.min_grid_x * self.cell_size + self.layout_width_px / 2.0)
        self.center_y_px = (self.min_grid_y * self.cell_size + self.layout_height_px / 2.0)

        # --- Core 3JS Components ---
        target_aspect = self.fig_width / self.fig_height

        margin_factor = 1.1
        content_width = self.layout_width_px * margin_factor
        content_height = self.layout_height_px * margin_factor

        if content_height <= 0:
             cam_h = 100 
             cam_w = cam_h * target_aspect
        else:
            cam_h = content_height / 2.0
            cam_w = cam_h * target_aspect

        # Define camera bounds symmetrically around (0,0)
        self.camera = three.OrthographicCamera(
            left=-cam_w,
            right=cam_w,
            top=cam_h,
            bottom=-cam_h,
            near=0.1,
            far=1000
        )
        # Position camera at origin looking down Z axis
        self.camera.position = [0, 0, 10]
        self.camera.lookAt([0, 0, 0])

        self.scene = three.Scene(background='lightgray')
        self.scene.add(three.AmbientLight(color='#FFFFFF', intensity=1.0))

        # --- Materials ---
        self._create_materials()

        # --- Build 2D Model ---
        self.facelet_meshes = {}
        self.cube_group = self._build_cube_model()
        self.scene.add(self.cube_group)

        # Move the entire group so its calculated center is at the origin (0,0)
        self.cube_group.position = [-self.center_x_px, -self.center_y_px, 0]


        # --- Renderer ---
        self.renderer = three.Renderer(camera=self.camera, scene=self.scene,
                                       controls=[], width=self.fig_width, height=self.fig_height,
                                       antialias=True)

        # --- GUI Elements ---
        self._create_gui()
        self._setup_callbacks()

        # --- Initial State Update ---
        self.update_visual_state()
        self.update_history()
        self.solution_text.value = ""

        # --- Display ---
        display(self.ui)

    def _create_materials(self):
        self.materials = {
            "White":  three.MeshBasicMaterial(color='#FFFFFF', side='DoubleSide'),
            "Yellow": three.MeshBasicMaterial(color='#FDFB45', side='DoubleSide'),
            "Blue":   three.MeshBasicMaterial(color='#4095FE', side='DoubleSide'),
            "Green":  three.MeshBasicMaterial(color='#00EF2A', side='DoubleSide'),
            "Red":    three.MeshBasicMaterial(color='#FF0000', side='DoubleSide'),
            "Orange": three.MeshBasicMaterial(color='#FE8800', side='DoubleSide'),
            "Black":  three.MeshBasicMaterial(color='#333333', side='DoubleSide'),
        }
        self.edges_material = three.LineBasicMaterial(color='#000000', linewidth=1)

    def _build_cube_model(self):
        main_group = three.Group()
        facelet_geom = three.PlaneGeometry(self.facelet_size, self.facelet_size)
        edges_geom = three.EdgesGeometry(facelet_geom)

        for direction, (base_grid_x, base_grid_y) in self.grid_positions.items():
             for r in range(3):
                 for c in range(3):
                    plot_grid_x = base_grid_x + c
                    plot_grid_y = base_grid_y + (2 - r)
                    center_x = (plot_grid_x + 0.5) * self.cell_size
                    center_y = (plot_grid_y + 0.5) * self.cell_size

                    mat = self.materials["Black"]
                    facelet_mesh = three.Mesh(facelet_geom, mat)
                    facelet_mesh.position = [center_x, center_y, 0]

                    edges = three.LineSegments(edges_geom, self.edges_material)
                    edges.position = facelet_mesh.position

                    tile_group = three.Group()
                    tile_group.add(facelet_mesh)
                    tile_group.add(edges)

                    main_group.add(tile_group)
                    self.facelet_meshes[(direction, r, c)] = facelet_mesh

        return main_group

    def update_visual_state(self):
        new_colors = self.colorizer.update_colors()
        face_id_maps = self.tracker.cube_current_faces_with_ids

        for (direction, r, c), facelet_mesh in self.facelet_meshes.items():
            try:
                id_array = face_id_maps.get(direction)
                if id_array is None: continue
                piece_id = id_array[r, c]
                piece_color_list = new_colors.get(piece_id)
                if piece_color_list is None: continue

                color_idx = self.colorizer.direction__color_idx_map.get(direction)
                if color_idx is not None and 0 <= color_idx < len(piece_color_list):
                    color_name = piece_color_list[color_idx]
                    new_material = self.materials.get(color_name, self.materials["Black"])
                    facelet_mesh.material = new_material
                else:
                    facelet_mesh.material = self.materials["Black"]
            except Exception as e:
                    print(f"Error updating facelet ({direction},{r},{c}): {type(e).__name__} - {e}")

    def _create_gui(self):
        moves_f = ['F', 'F2', 'F\'']
        moves_b = ['B', 'B2', 'B\'']
        moves_u = ['U', 'U2', 'U\'']
        moves_d = ['D', 'D2', 'D\'']
        moves_l = ['L', 'L2', 'L\'']
        moves_r = ['R', 'R2', 'R\'']
        all_moves = moves_f + moves_b + moves_u + moves_d + moves_l + moves_r

        # --- Create Buttons ---
        self.move_buttons = {}
        button_layout = widgets.Layout(width='40px', height='35px')
        for move in all_moves:
            button = widgets.Button(
                description=move, button_style='', tooltip=f'Apply {move} move',
                layout=button_layout
            )
            self.move_buttons[move] = button

        self.scramble_button = widgets.Button(
            description='Scramble', button_style='info', tooltip='Randomly scramble the cube',
            layout=widgets.Layout(width='auto'))
        self.reset_button = widgets.Button(
            description='Reset', button_style='warning', tooltip='Reset to solved state',
            layout=widgets.Layout(width='auto'))
        self.solve_button = widgets.Button(
            description='Solve', button_style='success', tooltip='Solve the cube (Not Implemented)',
            layout=widgets.Layout(width='auto'))
        self.apply_button = widgets.Button(
            description='Apply', tooltip='Apply custom move sequence',
            layout=widgets.Layout(width='auto'))

        # --- Create Text Fields ---
        textbox_height = '40px'
        self.history_text = widgets.Textarea(
            value='', placeholder='Move history appears here', disabled=True,
            layout=widgets.Layout(width='95%', height=textbox_height, overflow_y='auto')
        )
        self.solution_text = widgets.Textarea(
            value='', placeholder='Solver output appears here', disabled=True,
            layout=widgets.Layout(width='95%', height=textbox_height, overflow_y='auto', margin='5px 0 0 0')
        )
        self.move_input = widgets.Text(
            placeholder='Enter sequence (e.g., F R2 U\')',
            layout=widgets.Layout(flex='1 1 auto', width='auto')
        )

        # --- Arrange Layout ---
        action_button_box = widgets.HBox(
            [self.scramble_button, self.reset_button, self.solve_button],
            layout=widgets.Layout(justify_content='space-around')
        )
        left_panel = widgets.VBox(
            [action_button_box, self.history_text, self.solution_text],
            layout=widgets.Layout(min_width='250px', margin='0 15px 0 0')
        )

        move_button_rows = [
            widgets.HBox([self.move_buttons[m] for m in moves_f]),
            widgets.HBox([self.move_buttons[m] for m in moves_b]),
            widgets.HBox([self.move_buttons[m] for m in moves_u]),
            widgets.HBox([self.move_buttons[m] for m in moves_d]),
            widgets.HBox([self.move_buttons[m] for m in moves_l]),
            widgets.HBox([self.move_buttons[m] for m in moves_r])
        ]
        move_button_box = widgets.VBox(move_button_rows, layout=widgets.Layout(align_items='center')) # Center buttons

        custom_move_box = widgets.HBox(
            [self.move_input, self.apply_button],
            layout=widgets.Layout(width='100%')
            )
        
        right_panel = widgets.VBox(
            [move_button_box, custom_move_box],
            layout=widgets.Layout(min_width='150px')
            )

        self.control_panel = widgets.VBox(
            [left_panel, right_panel],
            layout=widgets.Layout(margin='10px 0 0 0', justify_content='space-between')
            )

        self.ui = widgets.HBox([self.renderer, self.control_panel])


    def _setup_callbacks(self):
        def make_move_callback(move_code):
            def on_click(button):
                self.apply_move(move_code)
            return on_click

        for move_code, button in self.move_buttons.items():
            button.on_click(make_move_callback(move_code))

        self.scramble_button.on_click(self.scramble_cube)
        self.reset_button.on_click(self.reset_cube)
        self.solve_button.on_click(self.solve_cube)
        self.apply_button.on_click(lambda b: self.apply_custom_moves())

    def apply_move(self, move):
        try:
            self.tracker.apply_moves(move)
            self.update_visual_state()
            self.update_history()
            self.solution_text.value = ""
        except ValueError as e:
            # Catch errors from the tracker's validation
            print(f"Error applying move {move}: {e}")
            original_placeholder = 'Enter sequence (e.g., F R2 U\')'
            self.move_input.placeholder = f"Error: {e}"
            def reset_placeholder():
                self.move_input.placeholder = original_placeholder
            threading.Timer(2.5, reset_placeholder).start()

        except Exception as e:
             print(f"An unexpected error occurred applying move {move}: {e}")

    def apply_custom_moves(self):
        moves_str = self.move_input.value.strip()
        error_msg = ""
        if not moves_str: return

        try:
            # Let the tracker handle parsing and validation
            self.tracker.apply_moves(moves_str)
            self.update_visual_state()
            self.update_history()
            self.solution_text.value = ""
            self.move_input.value = "" # Clear input on success
            self.move_input.placeholder = 'Enter sequence (e.g., F R2 U\')'
        except ValueError as e:
            # Catch parsing/validation errors from the tracker
            error_msg = f"Invalid sequence: {e}"
        except Exception as e:
            error_msg = f"Error applying sequence: {e}"

        if error_msg:
            # Display error message temporarily in the placeholder
            original_placeholder = 'Enter sequence (e.g., F R2 U\')'
            self.move_input.placeholder = error_msg
            # Define function to clear the message later
            def reset_placeholder():
                 if self.move_input.placeholder == error_msg:
                     self.move_input.placeholder = original_placeholder
            # Set timer to clear it
            threading.Timer(3.0, reset_placeholder).start()


    def scramble_cube(self, button):
        def remove(lst, item):
            return [x for x in lst if x != item]
        scramble_len = 20 # Standard scramble length
        axes = list(self.scramble_move_map.keys())
        scramble_moves_list = []
        current_axis = random.choice(axes) # Start with a random axis

        for _ in range(scramble_len):
            # Choose a move from the current axis's allowed moves (using HTM notation now)
            next_move = random.choice(self.scramble_move_map[current_axis])
            scramble_moves_list.append(next_move)
            # Choose the next axis, ensuring it's different from the current one
            current_axis = random.choice(remove(axes, current_axis))

        # Join the list of moves into a single string for the tracker
        scramble_str = "".join(scramble_moves_list)

        try:
            self.tracker.apply_moves(scramble_str)
            self.update_visual_state()
            self.update_history()
            self.solution_text.value = ""
        except ValueError as e:
            print(f"Error applying generated scramble '{scramble_str}': {e}")
        except Exception as e:
            print(f"Unexpected error during scramble application: {e}")


    def reset_cube(self, button):
        try:
            # Re-initialize Colorizer (which re-initializes Tracker)
            self.colorizer = CubeColorizer()
            self.tracker = self.colorizer.cube_tracker
            # Update visuals and history
            self.update_visual_state()
            self.update_history()
            self.solution_text.value = ""
        except Exception as e:
             print(f"Error resetting cube: {e}")

    def solve_cube(self, button):
        self.solution_text.value = "Solver not yet implemented!"
        def clear_solution_placeholder():
             if self.solution_text.value == "Solver not yet implemented!":
                 self.solution_text.value = ""
        threading.Timer(2.5, clear_solution_placeholder).start()

    def update_history(self):
        history = ' '.join(self.tracker.move_history)
        self.history_text.value = history

visualizer_2d = CubeVisualizer2D_pythreejs(figsize=(700,450))